In [ ]:
import pandas as pd
import numpy as np
from sklearn.model_selection import train_test_split, cross_val_score
from sklearn.preprocessing import StandardScaler
from sklearn.neighbors import KNeighborsClassifier
from sklearn.model_selection import cross_val_predict
from sklearn.metrics import accuracy_score
from sklearn.model_selection import GridSearchCV


In [2]:
data = pd.read_csv('/Users/vaibhavrangan/Downloads/Stat_303-3/Datasets/diabetes.csv')

data.head()

,Pregnancies,Glucose,BloodPressure,SkinThickness,Insulin,BMI,DiabetesPedigreeFunction,Age,Outcome
0,6,148,72,35,0,33.6,0.627,50,1
1,1,85,66,29,0,26.6,0.351,31,0
2,8,183,64,0,0,23.3,0.672,32,1
3,1,89,66,23,94,28.1,0.167,21,0
4,0,137,40,35,168,43.1,2.288,33,1


In [3]:
# Question 1
predictors = data[['Pregnancies', 'Glucose', 'BloodPressure', 'SkinThickness', 'Insulin', 'BMI', 'DiabetesPedigreeFunction', 'Age']]
y = data['Outcome']

X_train, X_test, y_train, y_test = train_test_split(predictors, y, test_size=0.2, random_state=12, stratify=y)

In [4]:
# Question 2
scaler = StandardScaler()
X_train_scaled = scaler.fit_transform(X_train)
X_test_scaled = scaler.transform(X_test)


In [5]:
# Question 3
model = KNeighborsClassifier()

grid = {'n_neighbors': np.arange(1, 60, 2), 'weights': ['uniform', 'distance']}

gscv = GridSearchCV(model, grid, cv=5, scoring='accuracy')

gscv.fit(X_train_scaled, y_train)

print(gscv.best_params_)

{'n_neighbors': np.int64(17), 'weights': 'uniform'}


In [7]:
# Question 4
cv_probs = cross_val_predict(gscv.best_estimator_, X_train_scaled, y_train, cv=5, method='predict_proba')[:, 1]
thresholds = np.arange(0, 1.01, 0.01)

accs = []

for thr in thresholds:
    cv_preds = cv_probs > thr
    acc = accuracy_score(y_train, cv_preds)
    accs.append(acc)

best_acc = np.array(accs).max()
best_threshold = thresholds[np.array(accs).argmax()]

print(best_acc, best_threshold)

0.758957654723127 0.48


In [ ]:
# Question 5

test_probs = gscv.best_estimator_.predict_proba(X_test_scaled)[:, 1]
test_preds = test_probs > best_threshold

test_acc = accuracy_score(y_test, test_preds)

print(test_acc)

0.7207792207792207
